# The Price is Right


1. This notebook: create a RAG database with our 400,000 training data

In [1]:
# imports

import os
import re
import math
import json
from tqdm import tqdm
import random
from dotenv import load_dotenv
from huggingface_hub import login
import numpy as np
import pickle
from sentence_transformers import SentenceTransformer
from datasets import load_dataset
import chromadb
from items import Item
from sklearn.manifold import TSNE
import plotly.graph_objects as go

In [2]:
# environment

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN', 'your-key-if-not-using-env')
DB = "products_vectorstore"

In [3]:
# Log in to HuggingFace

hf_token = os.environ['HF_TOKEN']
login(hf_token, add_to_git_credential=True)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [4]:
# With train.pkl in this folder, you can run this:

with open('train.pkl', 'rb') as file:
    train = pickle.load(file)

In [5]:
train[0].prompt

'How much does this cost to the nearest dollar?\n\nDelphi FG0166 Fuel Pump Module\nDelphi brings 80 years of OE Heritage into each Delphi pump, ensuring quality and fitment for each Delphi part. Part is validated, tested and matched to the right vehicle application Delphi brings 80 years of OE Heritage into each Delphi assembly, ensuring quality and fitment for each Delphi part Always be sure to check and clean fuel tank to avoid unnecessary returns Rigorous OE-testing ensures the pump can withstand extreme temperatures Brand Delphi, Fit Type Vehicle Specific Fit, Dimensions LxWxH 19.7 x 7.7 x 5.1 inches, Weight 2.2 Pounds, Auto Part Position Unknown, Operation Mode Mechanical, Manufacturer Delphi, Model FUEL PUMP, Dimensions 19.7\n\nPrice is $227.00'

# Now create a Chroma Datastore

In Week 5, we created a Chroma datastore with 123 documents representing chunks of objects from our fictional company Insurellm.

Now we will create a Chroma datastore with 400,000 products from our training dataset! It's getting real!

Note that we won't be using LangChain, but the API is very straightforward and consistent with before.

In [6]:
client = chromadb.PersistentClient(path=DB)

In [7]:
# Check if the collection exists and delete it if it does
collection_name = "products"
existing_collection_names = [collection.name for collection in client.list_collections()]
if collection_name in existing_collection_names:
    client.delete_collection(collection_name)
    print(f"Deleted existing collection: {collection_name}")

collection = client.create_collection(collection_name)

# Introducing the SentenceTransfomer

The all-MiniLM is a very useful model from HuggingFace that maps sentences & paragraphs to a 384 dimensional dense vector space and is ideal for tasks like semantic search.

https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2

It can run pretty quickly locally.

Last time we used OpenAI embeddings to produce vector embeddings. Benefits compared to OpenAI embeddings:
1. It's free and fast!
3. We can run it locally, so the data never leaves our box - might be useful if you're building a personal RAG


In [8]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [9]:
# Pass in a list of texts, get back a numpy array of vectors

vector = model.encode(["Well hi there"])[0]

In [10]:
vector

array([-9.46715623e-02,  4.27619517e-02,  5.51619940e-02, -5.10977814e-04,
        1.16203167e-02, -6.80130422e-02,  2.76404936e-02,  6.06974065e-02,
        2.88529955e-02, -1.74127873e-02, -4.94346134e-02,  2.30992399e-02,
       -1.28614800e-02, -4.31402475e-02,  2.17509978e-02,  4.26548868e-02,
        5.10500483e-02, -7.79727474e-02, -1.23247214e-01,  3.67455222e-02,
        4.54117730e-03,  9.47938412e-02, -5.53098321e-02,  1.70641020e-02,
       -2.92873029e-02, -4.47124317e-02,  2.06784457e-02,  6.39320463e-02,
        2.27428246e-02,  4.87789623e-02, -2.33503687e-03,  4.72859442e-02,
       -2.86258887e-02,  2.30624359e-02,  2.45131366e-02,  3.95681225e-02,
       -4.33176123e-02, -1.02316633e-01,  2.79871817e-03,  2.39304006e-02,
        1.61556266e-02, -8.99075996e-03,  2.07255092e-02,  6.40122592e-02,
        6.89179078e-02, -6.98360726e-02,  2.89759669e-03, -8.10990036e-02,
        1.71123017e-02,  2.50657834e-03, -1.06529079e-01, -4.87732552e-02,
       -1.67762470e-02, -

In [11]:
def description(item):
    text = item.prompt.replace("How much does this cost to the nearest dollar?\n\n", "")
    return text.split("\n\nPrice is $")[0]

In [12]:
description(train[0])

'Delphi FG0166 Fuel Pump Module\nDelphi brings 80 years of OE Heritage into each Delphi pump, ensuring quality and fitment for each Delphi part. Part is validated, tested and matched to the right vehicle application Delphi brings 80 years of OE Heritage into each Delphi assembly, ensuring quality and fitment for each Delphi part Always be sure to check and clean fuel tank to avoid unnecessary returns Rigorous OE-testing ensures the pump can withstand extreme temperatures Brand Delphi, Fit Type Vehicle Specific Fit, Dimensions LxWxH 19.7 x 7.7 x 5.1 inches, Weight 2.2 Pounds, Auto Part Position Unknown, Operation Mode Mechanical, Manufacturer Delphi, Model FUEL PUMP, Dimensions 19.7'

In [13]:
for i in tqdm(range(0, len(train), 1000)):
    documents = [description(item) for item in train[i: i+1000]]
    vectors = model.encode(documents).astype(float).tolist()
    metadatas = [{"category": item.category, "price": item.price} for item in train[i: i+1000]]
    ids = [f"doc_{j}" for j in range(i, i+1000)]
    collection.add(
        ids=ids,
        documents=documents,
        embeddings=vectors,
        metadatas=metadatas
    )

100%|██████████████████████████████████████████████████████████████████████████████| 400/400 [6:40:42<00:00, 60.11s/it]


In [15]:
print("Success!!")

Success!!
